# Create PDF

In [109]:
from reportlab.platypus import Table, SimpleDocTemplate, TableStyle, Paragraph, Image, PageBreak
from reportlab.lib import colors
from reportlab.lib import utils
from reportlab.lib.units import cm

from os import listdir

import csv

Definitions

In [110]:
# makeTable takes data and creates a table with border and grid
def makeTable(data):
    table = Table(data)
    table.setStyle(TableStyle([('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),('BOX', (0,0), (-1,-1), 0.75, colors.black)]))
    return table

In [111]:
# https://stackoverflow.com/questions/5327670/image-aspect-ratio-using-reportlab-in-python
def get_image(path, width=1*cm):
    img = utils.ImageReader(path)
    iw, ih = img.getSize()
    aspect = ih / float(iw)
    return Image(path, width=width, height=(width * aspect))

In [112]:
# Pagenumber:
# https://code.activestate.com/recipes/546511-page-x-of-y-with-reportlab/
# https://code.activestate.com/recipes/576832/

# Header:
# https://stackoverflow.com/questions/49344094/reportlab-pass-custom-argument-to-canvas#49350581
# (specifically Barranka's answer)

from reportlab.pdfgen import canvas
from reportlab.lib.units import mm

class NumberedCanvas(canvas.Canvas):
    def __init__(self, *args, **kwargs):
        canvas.Canvas.__init__(self, *args, **kwargs)
        self._saved_page_states = []
        self.header = ID_global

    def showPage(self):
        self._saved_page_states.append(dict(self.__dict__))
        self._startPage()

    def save(self):
        """add page info to each page (page x of y)"""
        num_pages = len(self._saved_page_states)
        for state in self._saved_page_states:
            self.__dict__.update(state)
            self.draw_page_number(num_pages)
            self.draw_header()
            canvas.Canvas.showPage(self)
        canvas.Canvas.save(self)

    def draw_page_number(self, page_count):
        self.setFont("Helvetica", 7)
        self.drawRightString(200*mm, 20*mm,
            "Page %d of %d" % (self._pageNumber, page_count))

    def draw_header(self):
        self.setFont("Helvetica", 12)
        self.drawString(100*mm, 280*mm, self.header)

General approach:
- Create document (doc)
- Create empty list (elements)
- Append to list
- Build doc

We can therefor make a forloop over the participants, fetch the relevant data and create a pdf for each participant. As of now, we use a handselected ID. We also have to move png and csv files manually, but this would be easy to change later.

In [113]:
# Simple exampel to illustrate usage

#doc = SimpleDocTemplate('test.pdf', rightMargin=72, leftMargin=72)
#elements = []
#
#data = [['a','b','c','d'],['1','2','3','4']]
#elements.append(makeTable(data))
#
#elements.append(Image('testimg.png'))
#
#doc.build(elements)

How to use:
- Moved output png and csv of correctionplot.ipynb from plots folder into pdf folder.
- DONE

In [114]:
datafolder = '../data/'

png_width_full = 20
png_width_lap = 19

for ID in listdir(datafolder):
    global ID_global 
    ID_global = ID
    output_location_string = datafolder+ID+'/'
    doc = SimpleDocTemplate('report{}.pdf'.format(ID), rightMargin=72, leftMargin=72)
    elements = []

    #Add full race graph
    elements.append(get_image(output_location_string+'fullracepd{}.png'.format(ID), width=png_width_full*cm))

    #Add table
    with open(output_location_string+'correlation_{}.csv'.format(ID)) as c:
        data = list(csv.reader(c))
        elements.append(makeTable(data))
    
    #Next page
    elements.append(PageBreak())

    #Find all lap plots
    keyword = 'lap'
    lap_plot_list = []
    for lap_plot_file in listdir(output_location_string):
        if keyword in lap_plot_file:
            lap_plot_list.append(lap_plot_file)

#    #Adjust lap graph size
#    if len(lap_plot_list) >= 4:
#        png_width_lap = 20 #at 4 or more graphs, report will be more than 2 pages

    #Sort and add graphs
    lap_plot_list.sort()
    for lap_graph in lap_plot_list:
        elements.append(get_image(output_location_string+lap_graph, width=png_width_lap*cm))

    doc.build(elements, canvasmaker=NumberedCanvas)